# Проектная работа 

## Введение в машинное обучение

<div class="alert alert-block alert-info">
<b>Цель проекта:</b> построить модель, которая сможет проанализировать поведение клиентов и предложить пользователям тариф "Смарт" или "Ультра". 

<b>Входные данные:</b> данные о повведении клиентов, которые уже перешли на эти тарифы.

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc">
<ul class="toc-item">
    <li><span><a href="#Изучение-данных-из-файла" data-toc-modified-id="Изучение-данных-из-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла</a></span></li>
     <li><span><a href="#Деление-данных" data-toc-modified-id="Деление-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Деление данных</a></span></li>
    <li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Построение моделей</a></span></li>
    <li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li>
    <li><span><a href="#Вывод" data-toc-modified-id="Вывод"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li>
    
</ul>    
    
</div>

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix

## Изучение данных из файла

In [2]:
pth1 = 'datasets/users_behavior.csv'
pth2 = '/datasets/users_behavior.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something os wrong')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<div class="alert alert-block alert-info">
Видим, что данные уже предобработаны, пропуски отсутствуют. 
    
Поля calls, minutes, mb_used являются признаками, поле is_ultra - это целевой признак, означающий принадлежность к тарифу ultra.    
    
При построении модели необходимо решить задачу бинарной (двоичной) классификации и произвести обучение с учителем.
</div>

## Деление данных

In [5]:
X = data.drop(['is_ultra'], axis=1)
y = data['is_ultra']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=12345)

In [7]:
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, random_state=12345)

<div class="alert alert-block alert-info">
Разделили выборку на обучающую, валидационную и тестовую. Принято делить в соотношения 3:1:1. Поэтому сначала выделяем под обучающую 60% набора данных, затем остаток делим на 2 части-валидационную и тестовую.
</div>

## Построение моделей

### Решающее дерево

In [8]:
#В этот словарь будем записывать модели и значение их accuracy на валидационной выборке
best_models = {}
#фиксируем модель решающего дерева без параметров
model1_tree = DecisionTreeClassifier(random_state=12345)
#обучаем
model1_tree.fit(x_train, y_train)
#Предсказываем
answers = model1_tree.predict(x_valid)
print(f"Accuracy на валидационной выборке: {accuracy_score(y_valid, answers)}")
best_models[model1_tree] = accuracy_score(y_valid, answers)

Accuracy на валидационной выборке: 0.7309486780715396


In [9]:
#добавим гиперпараметр max_depth, выберем модель с наилучшим accuracy в зависимости от значения глубины дерева
best_model = None
best_result = 0
best_depth = 0
for depth in range(1,10):
    model2_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model2_tree.fit(x_train, y_train)
    answers = model2_tree.predict(x_valid)
    result = accuracy_score(y_valid, answers)
    if result > best_result:
        best_model = model2_tree
        best_result = result
        best_depth = depth
        
best_models[best_model] = best_result    
print(f'Accuracy на валидационной выборке с max_depth, равным {best_depth} : {best_result}')

Accuracy на валидационной выборке с max_depth, равным 7 : 0.7993779160186625


### Случайный лес

In [10]:
#обучаем модель случайного леса без параметров
model1_forest = RandomForestClassifier(random_state=12345)
model1_forest.fit(x_train, y_train)
answers = model1_forest.predict(x_valid)
print(f"Accuracy на валидационной выборке: {accuracy_score(y_valid, answers)}")
best_models[model1_forest] = accuracy_score(y_valid, answers)

Accuracy на валидационной выборке: 0.7853810264385692


In [11]:
#добавим гиперпараметр и выберем лучшую модель по accuracy:
best_model = None
best_result = 0
best_estimator = 0
for est in range(1,15):
    model2_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model2_forest.fit(x_train, y_train)
    answers = model2_forest.predict(x_valid)
    result = accuracy_score(y_valid, answers)
    if result > best_result:
        best_model = model2_forest
        best_result = result
        best_estimator = est
        
best_models[best_model] = best_result         
print(f'Accuracy на валидационной выборке с n_estimators, равным {best_estimator}: {best_result}')        

Accuracy на валидационной выборке с n_estimators, равным 8: 0.7869362363919129


### Логистическая регрессия

In [12]:
#Обучим модель без параметров
model1_regression = LogisticRegression(random_state=12345)
model1_regression.fit(x_train, y_train)
answers = model1_regression.predict(x_valid)
print(f"Accuracy равно {accuracy_score(y_valid, answers)}")
best_models[model1_regression] = accuracy_score(y_valid, answers)

Accuracy равно 0.6842923794712286


In [13]:
#Обучим модель с гиперпараметрами
model2_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model2_regression.fit(x_train, y_train)
answers = model1_regression.predict(x_valid)
print(f"Accuracy равно {accuracy_score(y_valid, answers)}")
best_models[model2_regression] = accuracy_score(y_valid, answers)

Accuracy равно 0.6842923794712286


In [14]:
max_value = max(best_models.values())

for key, value in best_models.items():
    if value == max_value:
        print(f"Наилучшая модель: {key}")
        print(f"со значением accuracy, равным {value}")

Наилучшая модель: DecisionTreeClassifier(max_depth=7, random_state=12345)
со значением accuracy, равным 0.7993779160186625


<div class="alert alert-block alert-info">
    
**Вывод:** обучили несколько моделей, наилучшее качество по accuracy показала модель решающего дерева с максимальной глубиной, равной 7.


## Проверка на тестовой выборке

In [15]:
best_model_tree = DecisionTreeClassifier(random_state=12345, max_depth=7)
best_model_tree.fit(x_train, y_train)
predictions = best_model_tree.predict(x_test)
accuracy_score(y_test, predictions)

0.7822706065318819

## Дополнительное задание

In [16]:
#Построим константное предсказание
dc_mf = DummyClassifier()
dc_mf.fit(x_train, y_train)

DummyClassifier()

In [17]:
y_true = y_test
y_pred = dc_mf.predict(x_test)
dc_mf_tn, dc_mf_fp, dc_mf_fn, dc_mf_tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel() 

In [18]:
(dc_mf_tn + dc_mf_tp) / (dc_mf_tn + dc_mf_fp + dc_mf_fn + dc_mf_tp)

0.7060653188180405

In [19]:
accuracy_score(y_true, y_pred)

0.7060653188180405

<div class="alert alert-block alert-info">Для проверки модели на вменяемость построили базовую модель, точность предсказаний данной модели составила 71 %, что ниже, чем у выбранной наилучшей модели решающего дерева.

### Вывод 

<div class="alert alert-block alert-info">
В данной работе изучили данные, разделили данные на обучающую, тестовую, валидационную выборки. Настроили 3 модели на валидационной выборке: модель решающего дерева, случайного леса и логистической регрессии. После подбора гиперпараметров, было выявлено, что наилучшее качество на валидационной выборке показала модель решающего дерева с максимальной глубиной, равной 7. Эта модель была применена к тестовой выборке, точность предсказания составила 78%.


    
Также была построена константная модель и оценено ее качество при помощи такой метрики, как Accuracy. На тестовой выборке данная модель показала более низкое качество, чем модель решающего дерева.
    

  
Таким образом, для системы, рекомендующей пользователям тариф "Смарт" или "Утра", предлагается модель решающего дерева с максимальной глубиной, равной 7.
</div>